In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
import plotly.graph_objects as go
import plotly.express as px 
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from category_encoders.count import CountEncoder
from training_functions import *

# 0. CARGA DE DATAS.

In [2]:
path = '../outputs/'
df  = get_data(name_sav='df_HOMO_CLASS.sav',
               path=path)

__NOTA__: Se harán uso de algunas funciones presentes en el script __training_functions.py__. Se referirá a ellas con el siguiente estilo: *__estilo__*.

# 1. CONTEXTO DE LA MINICOMPETENCIA.

Estamos ante un problema de clasificación binaria con un evento de desbalanceo importante. 

Se hará una minicompetencia de forma arbitraria, en el sentido de que se considerará varios modelos "clásicos" y uno que es una versión más rápida de un Gradient Boosting. Estos son: `RandomForestClassifier`, `HistGradientBoostingClassifier`, `GradientBoostingClassifier`, `LogisticRegression`, `DecisionTreeClassifier`.

Los puntos generales a considerar en esta competencia son:
* Se usarán una semilla arbitraria (`seed = 5000`).

* Los modelos se entrenarán con los valores de sus hiperpárametros a como están por default en sus configuraciones desde `scikit-learn`. 

* Se dará prioridad al buen rendimiento del modelo con respecto a la clase minoritaria. Siendo así, se eligirá observar métricas que informen con respecto a esta clase.

* El mejor modelo será el que tenga un mejor performance en la métrica __f1__ de __cross-validation__, porque, al combinar métricas como el __recall__ y el __precision__, puede interpretarse como una especie de correlación, y entre más cercano a 1, se tendrá tanto un alto __precision__ como un alto __recall__, lo que implica una buena capacidad para clasificar correctamente las muestras positivas y minimizar los falsos positivos y falsos negativos. 

* Aunque el ganador lo dicidirá el __f1__, también se calculará la matriz de confusión de __testeo__, así como los scores de __precision__, __recall__, __coeficiente de correlación de Matthews (m_c)__, de __testeo__ y __cross-validation__, esto con fines informativos. Sin embargo, también pueden ayudar a elegir al modelo ganador en caso de empate.

* También, se calcularán las versiones "micro" de estas métricas, esto para ver vigilar la buena clasificación en ambas clases.

* Existen varios métodos para lidiar con el desbalanceo, entre submuestreos de la clase mayoritaria o sobremuestreos de la clase minoritaria. Aunque un método muy famoso de sobremuestreo es el `SMOTE`, en esta prueba se usará el método de submuestreo `RandomUnderSampler`. Esto es debido a que simplemente, por experiencia, se ha tenido mayor éxito en el pasado usando métodos de submuestreo.

* El subuestreo se hará de tal forma que se tenga un equilibrio total entre la clase minoritaria y la mayoritaria. Esto equivale a tener una estrategia de muestreo igual a 1 (`ratio_balance = 1`).

* Para la codificación de variables categóricas, se hará uso del`CountEncoder`, en su versión normalizada, el cual consiste en el porcentaje de representación que tiene cierta clase en una variable categórica.

* Se escalarán las variables numéricas para que tengan un rango común, sin cambiar su distribución.

* Se hará uso de __Pipilines__, es decir, de tuberias de proprocesamiento que contendrán pasos de transformación y de estimación necesarios.

* El uso de __Pipilines__ será especialemnte importante sobretodo en la etapa de __cross-validation k-folds__, para garatizar la repetición de los procesos de escalamiento, codificación, submuestreo y ajuste del modelo, en cada iteración-__fold__.

* Solo usaremos una cantidad pequeña de __folds__ (`k_folds = 3`).

* El proceso de __cross-validatión__ contempla una estrategia de división estratificada (`StratifiedKFold`). Esto con el fin de que en cada iteración-__fold__ la variable objetivo tenga aproximadamente la misma configuración de porcentajes de clase a como se tiene en el conjunto de __Entrenamiento__. De tal suerte que el proceso de submuetreo será aplicado en cada __fold__ y esto será posible con ayuda los __Pipilines__ ofrecidos por la librería `imbalanced-learn`.

# 2. CONFIGURACIONES GENERALES.

## 2.1. Valores y métricas

Listado de scores de interés:

In [3]:
scores = {'f1': 'f1',
          'f1_micro': 'f1_micro',
          'precision': 'precision',
          'precision_micro': 'precision_micro',
          'recall': 'recall',
          'recall_micro': 'recall_micro',
          'm_c': make_scorer(matthews_corrcoef)}

Se establecen los siguientes valores por default:

In [4]:
seed = 5000
ratio_balance = 1
k_folds = 3
verbose = 10
test_size = 0.25

Se establece el listado de features y la variable objetivo.

In [5]:
features_names =  ['FLAG_OWN_CAR',
                   'FLAG_OWN_REALTY',
                   'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE',
                   'NAME_FAMILY_STATUS',
                   'NAME_HOUSING_TYPE',
                   'OCCUPATION_TYPE',
                   'FLAG_WORK_PHONE',
                   'FLAG_PHONE',
                   'FLAG_EMAIL',
                   'AGE',
                   'CNT_CHILDREN',
                   'AMT_INCOME_TOTAL',
                   'WORK_YEARS']
objective_name = 'STATUS'

In [9]:
features = df[features_names]
label = df[objective_name]

In [10]:
numeric_names = list(features.select_dtypes(include=['float64', 'int64']).columns)
categorical_names = list(features.select_dtypes(include='object').columns)

In [ ]:
numeric_names

In [11]:
categorical_names

['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL']

Se establecen los conjuntos de  __entrenamiento__ y __testeo__.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    label,
                                                    random_state=seed,
                                                    test_size=test_size,
                                                    stratify=label)

Debido al uso de tranformaciones que afectan ya sea solo a variables categóricas o solo variables numéricas, el orden en que aparecen estos grupos de variables en los datasets es importante.

El orden en que van aplicandose la tranformaciones va corriendo a la derecha del dataset al grupo de variables al que se termina de aplicar dichas tranformaciones (ver la sección de Notes [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
.)

Para lidiar con esta situación se hace uso de la función *__get_feature_names_oder__*.

In [13]:
feature_names_order = get_feature_names_order(float_names=numeric_names,
                                              categorical_names=categorical_names)

In [14]:
X_train = X_train[feature_names_order]
X_test = X_test[feature_names_order]

## 2.2 Pipelines de preprocesamiento.

Se muestra el orden en que se aplicarán los procesos de tranformación y ajuste del modelo:
1. Preprocesamiento de variables numéricas: escalamiento.
2. Preprocesamiento de variables categóricas: codificación.
3. Submuestro.
4. Ajuste de modelo.

Los pasos 1 y 2 están cubiertos por las siguientes instancias:
```
numeric_transformer = Pipeline(steps=[('scaler',
                                       StandardScaler())])
categorical_transformer = Pipeline(steps=[('CountEncoder',
                                            CountEncoder(normalize=True))])
preprocessor = ColumnTransformer(remainder='passthrough',
                                 transformers=[('numeric',
                                                 numeric_transformer,
                                                 numeric_names),
                                               ('categorical',
                                                 categorical_transformer,
                                                 categorical_names)])
```
Las cuales serán proporcionadas por la función __*get_preprocessor*__.

Los pasos 3 y 4 serán cubiertos por el siguiente Pipeline, el cual cambiará dependiento el estimador:
```
transform = Pipeline(steps=[('processing',
                              preprocessor),
                            ('RandomUnderSampler',
                              RandomUnderSampler(random_state=seed,
                                                 sampling_strategy=ratio_balance)),
                            ('estimator',
                              model)])
```

# 3. COMPETENCIA DE MODELOS

Comienza la competencia de modelos. Se compararán las curvas __AUC-PR__ y las __F1 de testeo__.

In [25]:
models = [
          ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(random_state=seed)),
          ('RandomForest', RandomForestClassifier(random_state=seed)),
          ('GradientBoosting', GradientBoostingClassifier(random_state=seed)),
          ('LogisticRegression', LogisticRegression(random_state=seed)),
          ('DecisionTreeClassifier',  DecisionTreeClassifier(random_state=seed))]

In [26]:
pr_curves = []
f1_testeo_scores = []

# ITERAR SOBRE CADA MODELO Y CALCULAR LA CURVA PR
for model_name, model in models:
    print(f'NOMBRE: {model_name}')
    # CONFIGURACIONES GENERALES. ###########
    preprocessor = get_preprocessor(float_names=numeric_names,
                                    categorical_names=categorical_names)
    transform = Pipeline(steps=[('processing',
                                  preprocessor),
                                ('RandomUnderSampler',
                                  RandomUnderSampler(random_state=seed,
                                                     sampling_strategy=ratio_balance)),
                                ('estimator',
                                  model)])

    # AJUSTE DEL MODELO. ###################
    clf = transform.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1_testeo = f1_score(y_test, y_pred)
    f1_testeo_scores.append(f1_testeo)

    # OBTENER PROBABILIDADES ESTIMADAS #########
    probas_pred = clf.predict_proba(X_test)[:, 1]

    # CALCULAR LA CURVA PR Y EL ÁREA BAJO LA CURVA (AUC-PR) ##########
    precision, recall, _ = precision_recall_curve(y_test, probas_pred)
    auc_pr = auc(recall, precision)

    # GUARDAR LA CURVA PR Y ETIQUETA EN LA LISTA ####################
    pr_curves.append((recall,
                      precision,
                      f'{model_name} (AUC = {auc_pr:.2f})'))

NOMBRE: HistGradientBoostingClassifier
NOMBRE: RandomForest
NOMBRE: GradientBoosting
NOMBRE: LogisticRegression
NOMBRE: DecisionTreeClassifier


In [27]:
fig = go.Figure()
colors = px.colors.qualitative.Vivid

# AGREGAR TODAS LAS CURVAS PR
for i, (recall, precision, label) in enumerate(pr_curves):
    fig.add_trace(go.Scatter(x=recall,
                             y=precision,
                             mode='lines',
                             name=label,
                             line=dict(color=colors[i])))

In [28]:
fig.update_layout(
    title='Curvas PR de Modelos',
    xaxis=dict(title='Recall'),
    yaxis=dict(title='Precision'),
    legend=dict(x=1, y=1),
    showlegend=True,
    template='plotly_white'
)
# fig.show()

Se observa que el `RandomForest` tiene una mayor área bajo la curva PR. Es un indicador de que se comporta mejor en cuanto al precisión y recall de la clase minoritaria.

In [29]:
# LISTA DE MODELOS Y F1-SCORES
model_names = [model[0] for model in models]
colors = px.colors.qualitative.Vivid

# CREAR UNA GRÁFICA DE BARRAS
fig = go.Figure(data=[go.Bar(x=model_names,
                             y=f1_testeo_scores,
                             marker_color=colors)])

# PERSONALIZAR Y MOSTRAR LA GRÁFICA
fig.update_layout(title='Comparativa de Modelos',
                  xaxis_title='Modelos',
                  yaxis_title='F1-score',
                  yaxis=dict(range=[0, 1]))

fig.show()

Se observa que el `RandomForest` y el `DecisionTreeClassifier` tienen los mejores F1 de testeo. Se tomarán estos dos modelos como "finalistas" y se verá  se comportan sus métricas de testeo y cross-validation. Por medio del cross-validation, veremos que tan estables son estos modelos.

# 4. ENTRENAMIENTO DE DECISION TREEE CLASSIFIER.

Entrenemos el estimador `DecisionTreeClassifier`.

In [31]:
preprocessor = get_preprocessor(float_names=numeric_names,
                                categorical_names=categorical_names)
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  DecisionTreeClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['AGE', 'CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'WORK_YEARS']),
                                                 ('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_PHONE',
                                                   'FLAG_EMAIL'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator', DecisionTreeClassifier(random_state=5000))])

## 4. 1 Ajuste del modelo para obtener métricas de testeo.

In [32]:
clf = transform.fit(X_train, y_train)
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [34]:
f_1_testeo = f1_score(y_test, y_pred)
f_1_testeo_micro = f1_score(y_test, y_pred, average='micro')
precision_testeo = precision_score(y_test, y_pred)
precision_micro_testeo = precision_score(y_test, y_pred, average='micro')
recall_testeo = recall_score(y_test, y_pred)
recall_micro_testeo = recall_score(y_test, y_pred, average='micro')
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [35]:
print('f1-testeo:', round(f_1_testeo, 2))
print('f1-micro-testeo:', round(f_1_testeo_micro, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('precision-micro-testeo:', round(precision_micro_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('recall-micro-testo:', round(recall_micro_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.25
f1-micro-testeo: 0.6
precision-testeo: 0.15
precision-micro-testeo: 0.6
recall-testo: 0.65
recall-micro-testo: 0.6
m_c-testeo: 0.15
matriz de confusión de testeo [[667 451]
 [ 44  81]]


## 4.2 Cross-validation.

In [36]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.211) f1_micro: (test=0.530) m_c: (test=0.089) precision: (test=0.127) precision_micro: (test=0.530) recall: (test=0.629) recall_micro: (test=0.530) total time=   0.1s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.1s


[CV] END  f1: (test=0.230) f1_micro: (test=0.574) m_c: (test=0.123) precision: (test=0.140) precision_micro: (test=0.574) recall: (test=0.637) recall_micro: (test=0.574) total time=   0.1s
[CV] START .....................................................................
[CV] END  f1: (test=0.230) f1_micro: (test=0.565) m_c: (test=0.123) precision: (test=0.140) precision_micro: (test=0.565) recall: (test=0.648) recall_micro: (test=0.565) total time=   0.1s


Se obtienen las métricas de __cross-validation__.

In [38]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('f1-micro-cv:', round(cv_results['test_f1_micro'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('precision-micro-cv:', round(cv_results['test_precision_micro'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('recall-micro-cv:', round(cv_results['test_recall_micro'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.22
f1-micro-cv: 0.56
precision-cv: 0.14
precision-micro-cv: 0.56
recall-cv: 0.64
recall-micro-cv: 0.56
m_c-cv: 0.11


# 5. ENTRENAMIENTO DE RANDOM FOREST.

Entrenemos el estimador `RandomForestClassifier`.

In [40]:
preprocessor = get_preprocessor(float_names=numeric_names,
                                categorical_names=categorical_names)
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  RandomForestClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['AGE', 'CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'WORK_YEARS']),
                                                 ('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_PHONE',
                                                   'FLAG_EMAIL'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator', RandomForestClassifier(random_state=5000))])

## 5. 1 Ajuste del modelo para obtener métricas de testeo.

In [42]:
clf = transform.fit(X_train, y_train)
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [44]:
f_1_testeo = f1_score(y_test, y_pred)
f_1_testeo_micro = f1_score(y_test, y_pred, average='micro')
precision_testeo = precision_score(y_test, y_pred)
precision_micro_testeo = precision_score(y_test, y_pred, average='micro')
recall_testeo = recall_score(y_test, y_pred)
recall_micro_testeo = recall_score(y_test, y_pred, average='micro')
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [45]:
print('f1-testeo:', round(f_1_testeo, 2))
print('f1-micro-testeo:', round(f_1_testeo_micro, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('precision-micro-testeo:', round(precision_micro_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('recall-micro-testo:', round(recall_micro_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.29
f1-micro-testeo: 0.67
precision-testeo: 0.18
precision-micro-testeo: 0.67
recall-testo: 0.66
recall-micro-testo: 0.67
m_c-testeo: 0.21
matriz de confusión de testeo [[752 366]
 [ 42  83]]


## 5.2 Cross-validation.

In [46]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

Se obtienen las métricas de __cross-validation__.

In [48]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('f1-micro-cv:', round(cv_results['test_f1_micro'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('precision-micro-cv:', round(cv_results['test_precision_micro'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('recall-micro-cv:', round(cv_results['test_recall_micro'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.24
f1-micro-cv: 0.62
precision-cv: 0.15
precision-micro-cv: 0.62
recall-cv: 0.6
recall-micro-cv: 0.62
m_c-cv: 0.14


# 6. MODELO GANADOR.

El `RandomForestClassifier` es el modelo ganador ya que tiene un  __F1 de validation__ con 0.24 frente a 0.22 del `DecisionTreeClassifier`, además de que ha tenido una mejor métrica __AUC-PR__ (0.43 vs 0.41) y mejor __F1 de testeo__ (0.29 vs 0.25). 